# Inserting CITY DATAS Data To MySQL with SQLAlchemy

## Collecting data from the API and creating dataframe

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [8]:
def recreate_wiki(cities):
  # empty list that will be filled with one dictionary of information per city
  list_for_df = []
  
  # begin a for loop to create a dictionary of information for each city
  for city in cities:
    # we can use the universal nature of wikipedias urls to our advantage here
    # all of the urls are the same besides the city name
    url = f'https://en.wikipedia.org/wiki/{city}'

    # here we make our soup for the city
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # here we initialise our empty dictionary for the city
    response_dict = {}

    # here we fill the dictionary with information using the ids, classes, and selectors that we found in the html
    response_dict['city'] = soup.select(".firstHeading")[0].get_text()
    response_dict['country'] = soup.select(".infobox-data")[0].get_text()
    response_dict['latitude'] = soup.select(".latitude")[0].get_text()
    response_dict['longitude'] = soup.select(".longitude")[0].get_text()
    # not all of the wikipedia pages contain elevation, look at Hamburg
    # the if clause means that our code can continue and won't stop at this hurdle
    if soup.select_one('.infobox-label:-soup-contains("Elevation")'):
      response_dict['elevation'] = soup.select_one('.infobox-label:-soup-contains("Elevation")').find_next(class_='infobox-data').get_text()
    response_dict['website'] = soup.select_one('.infobox-label:-soup-contains("Website")').find_next(class_='infobox-data').get_text()
    if soup.select_one('th.infobox-header:-soup-contains("Population")'):
        response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
    
    # add our dictionary for the city to list_for_df
    list_for_df.append(response_dict)
  
  # make the DataFrame
  cities_df = pd.DataFrame(list_for_df)

  # fixing latitude
  cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  # fixing longitude
  cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  # fixing elevation
  cities_df.insert(4, 'elevation_in_meters', cities_df['elevation'].str.split('m').str[0].str.strip())

  # return the DataFrame
  return cities_df
list_of_cities = ['Berlin', 'Hamburg', 'London', 'Manchester', 'Barcelona']
city_data_df=recreate_wiki(list_of_cities)
city_data_df['city_id']=[1,2,3,4,5]
city_data_df=city_data_df.iloc[:,[8,0,1,2,3,4,5,6,7]]
city_data_df

,city_id,city,country,latitude,longitude,elevation_in_meters,elevation,website,population
0,1,Berlin,Germany,52.3112,13.2418,34,34 m (112 ft),berlin.de,"3,677,472"
1,2,Hamburg,Germany,53.33N,10.00E,NaN,NaN,hamburg.com,"1,906,411"
2,3,London,United Kingdom,51.3026,0.739,36 ft (11,36 ft (11 m),www.london.gov.uk,"8,799,800"
3,4,Manchester,United Kingdom,53.2846,2.1443,38,38 m (125 ft),manchester.gov.uk,"551,938"
4,5,Barcelona,Spain,41.23N,2.11E,12,12 m (39 ft),www.barcelona.cat,"1,620,343"


# SQLAlchemy

In [9]:
!pip install sqlalchemy 
!pip install pymysql
!pip install mysql-connector-python-rf
import sqlalchemy # install if needed
import pymysql
from sqlalchemy import create_engine
import mysql.connector
import socket

In [10]:
schema="gans"   # name of the database you want to use here
host="localhost"        # to connect to your local server
user="root"
password="Payment_123" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'  

In [11]:
socket.getaddrinfo('localhost', 8080)
cnx = create_engine(con,echo=False)  
df = pd.read_sql('SELECT * FROM city_data', con)  
df

,city_id,city,country,latitude,longitude,elevation_in_meters,elevation,website,population


In [12]:
city_data_df.to_sql('city_data', con=cnx, if_exists='append', index=False) 

5